# **Fact Orders**

**Data Reading**

In [0]:
df = spark.sql("Select * from databricks_cat.silver.orders_silver")
df.display()

In [0]:
df_dimCustomer = spark.sql("select DimCustomersKey, customer_id as dim_customer_id from databricks_cat.gold.dimcustomers")
df_dimProduct =  spark.sql("select product_id as DimProductsKey, product_id as dim_product_id from databricks_cat.gold.dimproducts_v2")

**Fact Dataframe**

In [0]:
df_fact = df.join(df_dimCustomer, df.customer_id == df_dimCustomer.dim_customer_id, "left").join(df_dimProduct, df.product_id == df_dimProduct.DimProductsKey, "left")
df_fact_new = df_fact.drop('dim_customer_id', 'dim_product_id', 'customer_id', 'product_id')
df_fact_new.display()

**Upsert on fact table**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_cat.gold.FactOrders"):
    dlt_obj = DeltaTable.forName(spark, "databricks_cat.gold.FactOrders")
    dlt_obj.alias("t")\
        .merge(df_fact_new.alias("s"), "t.order_id = s.order_id AND t.DimCustomersKey = s.DimCustomersKey AND t.DimProductsKey = s.DimProductsKey")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_fact_new.write.format("delta")\
        .option("path", "abfss://gold@fmcdatabrickse2e.dfs.core.windows.net/FactOrders")\
            .saveAsTable("databricks_cat.gold.FactOrders")


In [0]:
%sql
SELECT * FROM databricks_cat.gold.factorders